In [5]:
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.dates as mdates
import pandas as pd
import pandas_datareader.data as web
import yfinance as yf
from pandas_datareader import data as pdr
#import cufflinks as cf
#from plotly.offline import iplot, init_notebook_mode
#import chart_studio.plotly as py
#import mplfinance as mpf
#from mplfinance.original_flavor import candlestick_ohlc

import requests
import bs4
from bs4 import BeautifulSoup
from IPython.display import display
#import pandas as pd
import numpy as np
#from tqdm.notebook import tqdm
from datetime import datetime, timedelta, timezone
import re
import json
from typing import List, Dict, Tuple, Optional, Union, Any, Set
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle
import requests
import os

style.use('ggplot')

# 1) Using pandas datareader and Yahoo Finance
yf.pdr_override()

df = pdr.get_data_yahoo('AMZN', start = '2019-03-01', end='2024-03-10', progress=False)
df.to_csv('amzn.csv')

In [13]:
'''
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
resp = requests.get(url)
soup = BeautifulSoup(resp.content, 'html.parser')

table = soup.find('table', {'class':'wikitable sortable'})
tickers = []
for row in table.findAll('tr')[1:]:
  ticker = row.findAll('td')[0].text
  ticker = ticker.strip()                #to drop '\n' character 
  tickers.append(ticker)

with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)

print(tickers)

'''

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DH

In [2]:
def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = BeautifulSoup(resp.content, 'html.parser')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        ticker = ticker.strip()
        tickers.append(ticker)
    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)
    #print(tickers)
    return tickers

In [ ]:
#save_sp500_tickers()

In [46]:
def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = dt.datetime(2018, 1, 1)      #start date
    end = dt.datetime.now()              #End date which is current
    for ticker in tickers:
        # just in case your connection breaks, we'd like to save our progress!
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = pdr.get_data_yahoo(ticker, start = start, end= end, progress=False)
            #df = web.DataReader(ticker, 'yahoo', start, end)
            #df.reset_index(inplace=True)
            #df.set_index("Date", inplace=True)
            #df = df.drop("Symbol", axis=1)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))


get_data_from_yahoo()

Already have MMM
Already have AOS
Already have ABT
Already have ABBV
Already have ACN
Already have ADBE
Already have AMD
Already have AES
Already have AFL
Already have A



1 Failed download:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2018-01-01 00:00:00 -> 2024-03-13 21:43:14.815695)')


In [6]:
def compile_data():
    with open("sp500tickers.pickle", "rb") as f:
        tickers = pickle.load(f)

    main_df = pd.DataFrame()

    for count, ticker in enumerate(tickers):
        df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
        df.set_index('Date', inplace=True)

        df.rename(columns={'Adj Close': ticker}, inplace=True)
        df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1, inplace=True)

        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how='outer')      #Joining all the Adjusted Close columns of all the stocks

        if count % 2 == 0:
            print(count)
    print(main_df.tail())
    main_df.to_csv('sp500_joined_closes.csv')


compile_data()

0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
100
102
104
106
108
110
112
114
116
118
120
122
124
126
128
130
132
134
136
138
140
142
144
146
148
150
152
154
156
158
160
162
164
166
168
170
172
174
176
178
180
182
184
186
188
190
192
194
196
198
200
202
204
206
208
210
212
214
216
218
220
222
224
226
228
230
232
234
236
238
240
242
244
246
248
250
252
254
256
258
260
262
264
266
268
270
272
274
276
278
280
282
284
286
288
290
292
294
296
298
300
302
304
306
308
310
312
314
316
318
320
322
324
326
328
330
332
334
336
338
340
342
344
346
348
350
352
354
356
358
360
362
364
366
368
370
372
374
376
378
380
382
384
386
388
390
392
394
396
398
400
402
404
406
408
410
412
414
416
418
420
422
424
426
428
430
432
434
436
438
440
442
444
446
448
450
452
454
456
458
460
462
464
466
468
470
472
474
476
478
480
482
484
486
488
490
492
494
496
498
500
502
                  MMM        AOS         ABT   

In [7]:
def visualize_data():
    df = pd.read_csv('sp500_joined_closes.csv')
    # Convert the 'Date' column to datetime objects
    df['Date'] = pd.to_datetime(df['Date'])
    # Convert the 'Date' column to Unix timestamps
    df['Date'] = df['Date'].apply(lambda x: x.timestamp())

    # Convert the 'Date' column to float
    df['Date'] = df['Date'].astype(float)

    # Print the DataFrame
    print(df.head())
    
    #df = df.reset_index()
    #df['Date'] = df['Date'].map(mdates.date2num)
    print(df.dtypes)
    print(df.corr())
   # df['ABBV'].plot()
   # plt.show()
visualize_data()

           Date         MMM        AOS        ABT       ABBV         ACN  \
0  1.514851e+09  184.653259  55.089031  52.831726  74.280685  140.376053   
1  1.514938e+09  184.645401  55.420311  52.948547  75.443100  141.023865   
2  1.515024e+09  187.058945  55.679947  52.858696  75.012840  142.693771   
3  1.515110e+09  188.516510  56.396206  53.011467  76.318672  143.870834   
4  1.515370e+09  187.905273  56.664799  52.858696  75.095879  145.020538   

         ADBE    AMD       AES        AFL  ...         WTW         GWW  \
0  177.699997  10.98  8.897790  37.835312  ...  134.813431  214.155701   
1  181.039993  11.55  8.889608  37.947144  ...  137.335617  212.811005   
2  183.220001  12.12  8.856895  38.338535  ...  138.729675  214.473633   
3  185.339996  11.88  8.889608  38.592281  ...  139.619354  216.463440   
4  185.039993  12.28  8.889608  38.691212  ...  138.867325  216.463440   

         WYNN        XEL        XYL        YUM        ZBRA         ZBH  \
0  153.166199  40.135132

In [9]:
def process_data_for_labels(ticker):
    hm_days = 7
    df = pd.read_csv('sp500_joined_closes.csv', index_col=0)
    tickers = df.columns.values.tolist()
    df.fillna(0, inplace=True)

    for i in range(1,hm_days+1):
        df['{}_{}d'.format(ticker,i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker]

    df.fillna(0, inplace=True)
    return tickers, df

#process_data_for_labels('AMD')

In [10]:
def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.02   #stock price changes by 2% we have to take notice with in 7 days
    for col in cols:
        if col > requirement:
            return 1
        if col < -requirement:
            return -1
    return 0

In [11]:
from collections import Counter

def extract_featuresets(ticker):
    tickers, df = process_data_for_labels(ticker)

    df['{}_target'.format(ticker)] = list(map( buy_sell_hold,
                                               df['{}_1d'.format(ticker)],
                                               df['{}_2d'.format(ticker)],
                                               df['{}_3d'.format(ticker)],
                                               df['{}_4d'.format(ticker)],
                                               df['{}_5d'.format(ticker)],
                                               df['{}_6d'.format(ticker)],
                                               df['{}_7d'.format(ticker)] ))
    vals = df['{}_target'.format(ticker)].values.tolist()
    str_vals = [str(i) for i in vals]
    print('Data spread:',Counter(str_vals))
    df.fillna(0, inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)
    df_vals = df[[ticker for ticker in tickers]].pct_change()    # % chage data for all of the companies - X_train
    df_vals = df_vals.replace([np.inf, -np.inf], 0)
    df_vals.fillna(0, inplace=True)
    X = df_vals.values                                    #Feature sets
    y = df['{}_target'.format(ticker)].values             #Lables  - Target classification
    return X,y,df


#extract_featuresets('XOM')

In [12]:
import sklearn as scikit_learn
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier



In [14]:
def do_ml(ticker):
    # Extract features and labels
    X, y, df = extract_featuresets(ticker)

    # Check if any of the returned values are None or empty
    if any(elem is None or (isinstance(elem, np.ndarray) and elem.size == 0) or (isinstance(elem, pd.DataFrame) and elem.empty) for elem in (X, y, df)):
        print("Error: extract_featuresets returned None or empty")
        return None

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    # Initialize the classifiers for the voting ensemble
    classifiers = [('LinearSVC', LinearSVC()), 
                   ('KNeighbors', KNeighborsClassifier()), 
                   ('RandomForest', RandomForestClassifier())]

    # Create the voting classifier
    clf = VotingClassifier(classifiers)

    # Train the model
    clf.fit(X_train, y_train)

    # Evaluate the model
    accuracy = clf.score(X_test, y_test)
    print('Accuracy:', accuracy)

    # Make predictions
    predictions = clf.predict(X_test)
    print('Predicted class counts:', Counter(predictions))

    return accuracy
# examples of running:
do_ml('XOM')
#do_ml('AAPL')
#do_ml('ABT')

Data spread: Counter({'1': 734, '-1': 596, '0': 229})


C:\Users\16475\anaconda3\envs\Foundations_Data_Science\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Accuracy: 0.49902912621359224
Predicted class counts: Counter({1: 331, -1: 182, 0: 2})


0.49902912621359224